In [ ]:
import websocket
import json
from urllib.parse import quote

In [ ]:
ws = websocket.WebSocket()
website_id='google.com'
ws.connect(f'wss://plankton-app-5rfza.ondigitalocean.app/websites?scraping={website_id}')
ws.recv()

In [ ]:
ws.close()

In [ ]:
ws = websocket.WebSocket()
page_id = 'https://betway.com/es/sports/evt/10682299","websiteId'
website_id='betway.com'
# uri = 'wss://plankton-app-5rfza.ondigitalocean.app'
uri='ws://localhost:8000'
ws.connect(f'{uri}/games/{website_id}?game_id={page_id}')
ws.recv()

In [8]:
from datetime import datetime, timezone
datetime.now(timezone.utc)

datetime.datetime(2022, 12, 10, 21, 2, 38, 819164, tzinfo=datetime.timezone.utc)

In [42]:
from core.models import r
from redis.commands.search.query import Query, NumericFilter
from  redis.commands.search import aggregation
from redis.commands.search import reducers

In [70]:
r.ft('idxGames').search(Query('20').sort_by('lastScraping', asc=True))

Result{3 total, docs: [Document {'id': 'games:betano.com:pe./cuotas-de-partido/ado-20-groene-ster/30370892/', 'payload': None, 'json': '{"id":"https://pe.betano.com/cuotas-de-partido/ado-20-groene-ster/30370892/","websiteId":"betano.com","urlSource":"https://pe.betano.com/sport/futbol/campeonatos/paises-bajos/11376/","sport":"futbol","fullName":"Ado 20 vs Groene Ster","firstTeam":"Ado 20","secoundTeam":"Groene Ster","scraping":false,"lastUpdate":"2022-12-10T21:13:33.019403+00:00"}'}, Document {'id': 'games:betano.com:pe./cuotas-de-partido/coleraine-sub-20-ballymena-united-sub-20/30236431/', 'payload': None, 'json': '{"id":"https://pe.betano.com/cuotas-de-partido/coleraine-sub-20-ballymena-united-sub-20/30236431/","websiteId":"betano.com","urlSource":"https://pe.betano.com/sport/futbol/campeonatos/irlanda-del-norte/11377/","sport":"futbol","fullName":"Coleraine Sub-20 vs Ballymena United Sub-20","firstTeam":"Coleraine Sub-20","secoundTeam":"Ballymena United Sub-20","scraping":false,"las

In [ ]:
from core.webdriver import init_driver
from selenium.webdriver.common.by import By
from core.models import Bet
from time import sleep
driver = init_driver()
url='https://betway.com/es/sports/evt/10682245'
driver.get(url)

In [ ]:
sleep(10)
status_code = driver.find_element(By.CSS_SELECTOR, 'meta[http-equiv="status"]').get_attribute('content')
print(status_code == '404')

In [ ]:
def get_bids(page_url):
    for elm in driver.find_elements(By.CSS_SELECTOR, '.collapsablePanel'):
        collapsed_elm = elm.find_element(By.CSS_SELECTOR, '.collapsableContent')
        if 'empty' in collapsed_elm.get_attribute('class'):
            try:
                elm.click()
            except:
                pass
        try:
            header_elm = elm.find_element(By.CSS_SELECTOR, '.titleText>.title')
            cashout_elm = elm.find_element(By.CSS_SELECTOR, '.cashOutMarketIndicatorContainer')
        except:
            continue    
        try:
            labels_elms = elm.find_elements(By.CSS_SELECTOR, '.outcomeHeader, .outcomeItemHeader')
            odds_elms = elm.find_elements(By.CSS_SELECTOR, '.oddsDisplay')
        except:
            continue
        
        for labels_elm, odds_elm in zip(labels_elms, odds_elms):
            try:
                group = header_elm.text.strip()
                name = labels_elm.text.strip()
                bet = Bet(
                    id=f'{group}-{name}'.replace(' ', '-').lower(),
                    websiteId='betway.com',
                    gameId=page_url,
                    group=group, 
                    name=name, 
                    bet=float(odds_elm.text.replace(',', '.'))
                )
            except Exception as e:
                print('Error', e)

In [ ]:
get_bids(url)